## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['NAME'])

In [3]:
# Determine the number of unique values in each column.
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

EIN 34299
APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts[counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classificationbinning = application_df['CLASSIFICATION'].value_counts()
classificationbinning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
countsclassification = classificationbinning[classificationbinning>1]
countsclassification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classificationbinning[classificationbinning<100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]
hidden_nodes1=7
hidden_nodes2=14
hidden_nodes3=21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 357       
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 21)                315       
                                                                 
 dense_3 (Dense)             (None, 1)                 22        
                                                                 
Total params: 806
Trainable params: 806
Non-trainable params: 0
_________________________________________________________________


2022-11-16 23:17:01.777147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=200)

Epoch 1/200
684/684 [==============================] - 1s 648us/step - loss: 0.5999 - accuracy: 0.7071 - val_loss: 0.5553 - val_accuracy: 0.7383
Epoch 2/200
684/684 [==============================] - 0s 540us/step - loss: 0.5642 - accuracy: 0.7248 - val_loss: 0.5490 - val_accuracy: 0.7375
Epoch 3/200
684/684 [==============================] - 0s 532us/step - loss: 0.5579 - accuracy: 0.7271 - val_loss: 0.5489 - val_accuracy: 0.7352
Epoch 4/200
684/684 [==============================] - 0s 534us/step - loss: 0.5559 - accuracy: 0.7272 - val_loss: 0.5454 - val_accuracy: 0.7391
Epoch 5/200
684/684 [==============================] - 0s 535us/step - loss: 0.5541 - accuracy: 0.7276 - val_loss: 0.5453 - val_accuracy: 0.7375
Epoch 6/200
684/684 [==============================] - 0s 530us/step - loss: 0.5531 - accuracy: 0.7288 - val_loss: 0.5445 - val_accuracy: 0.7318
Epoch 7/200
684/684 [==============================] - 0s 529us/step - loss: 0.5520 - accuracy: 0.7271 - val_loss: 0.5458 - val_ac

684/684 [==============================] - 0s 542us/step - loss: 0.5378 - accuracy: 0.7366 - val_loss: 0.5401 - val_accuracy: 0.7383
Epoch 58/200
684/684 [==============================] - 0s 546us/step - loss: 0.5384 - accuracy: 0.7349 - val_loss: 0.5397 - val_accuracy: 0.7391
Epoch 59/200
684/684 [==============================] - 0s 545us/step - loss: 0.5373 - accuracy: 0.7350 - val_loss: 0.5423 - val_accuracy: 0.7372
Epoch 60/200
684/684 [==============================] - 0s 539us/step - loss: 0.5371 - accuracy: 0.7359 - val_loss: 0.5402 - val_accuracy: 0.7391
Epoch 61/200
684/684 [==============================] - 0s 543us/step - loss: 0.5374 - accuracy: 0.7357 - val_loss: 0.5420 - val_accuracy: 0.7370
Epoch 62/200
684/684 [==============================] - 0s 546us/step - loss: 0.5372 - accuracy: 0.7370 - val_loss: 0.5408 - val_accuracy: 0.7385
Epoch 63/200
684/684 [==============================] - 0s 545us/step - loss: 0.5373 - accuracy: 0.7365 - val_loss: 0.5413 - val_accuracy

684/684 [==============================] - 0s 546us/step - loss: 0.5335 - accuracy: 0.7369 - val_loss: 0.5415 - val_accuracy: 0.7388
Epoch 114/200
684/684 [==============================] - 0s 548us/step - loss: 0.5335 - accuracy: 0.7371 - val_loss: 0.5424 - val_accuracy: 0.7388
Epoch 115/200
684/684 [==============================] - 0s 549us/step - loss: 0.5339 - accuracy: 0.7377 - val_loss: 0.5430 - val_accuracy: 0.7416
Epoch 116/200
684/684 [==============================] - 0s 547us/step - loss: 0.5333 - accuracy: 0.7378 - val_loss: 0.5441 - val_accuracy: 0.7378
Epoch 117/200
684/684 [==============================] - 0s 546us/step - loss: 0.5332 - accuracy: 0.7385 - val_loss: 0.5436 - val_accuracy: 0.7419
Epoch 118/200
684/684 [==============================] - 0s 546us/step - loss: 0.5332 - accuracy: 0.7383 - val_loss: 0.5437 - val_accuracy: 0.7378
Epoch 119/200
684/684 [==============================] - 0s 547us/step - loss: 0.5329 - accuracy: 0.7394 - val_loss: 0.5425 - val_ac

Epoch 169/200
684/684 [==============================] - 0s 547us/step - loss: 0.5307 - accuracy: 0.7409 - val_loss: 0.5456 - val_accuracy: 0.7383
Epoch 170/200
684/684 [==============================] - 0s 547us/step - loss: 0.5299 - accuracy: 0.7407 - val_loss: 0.5435 - val_accuracy: 0.7396
Epoch 171/200
684/684 [==============================] - 0s 544us/step - loss: 0.5300 - accuracy: 0.7418 - val_loss: 0.5436 - val_accuracy: 0.7396
Epoch 172/200
684/684 [==============================] - 0s 548us/step - loss: 0.5299 - accuracy: 0.7410 - val_loss: 0.5417 - val_accuracy: 0.7393
Epoch 173/200
684/684 [==============================] - 0s 546us/step - loss: 0.5303 - accuracy: 0.7421 - val_loss: 0.5436 - val_accuracy: 0.7424
Epoch 174/200
684/684 [==============================] - 0s 545us/step - loss: 0.5297 - accuracy: 0.7411 - val_loss: 0.5422 - val_accuracy: 0.7393
Epoch 175/200
684/684 [==============================] - 0s 546us/step - loss: 0.5293 - accuracy: 0.7419 - val_loss: 0

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5555 - accuracy: 0.7266 - 88ms/epoch - 327us/step
Loss: 0.5555251836776733, Accuracy: 0.7266472578048706


In [16]:
# Export our model to HDF5 file
nn.save('../AlphabetSoupCharity_Optimization.h5')